<a href="https://colab.research.google.com/github/satyams00/Chat-Bot/blob/main/CoverlessTextStenography.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from transformers import pipeline
import random

In [32]:
checkpoint = "MBZUAI/LaMini-Flan-T5-248M"

model = pipeline('text2text-generation', model = checkpoint)

text=["human body","science","earth","stars","animal","electronic acessories"]

generated_text =''

text = random.choice(text)
def get_cover_text():
    for i in range(0,10):
        global generated_text
        str = "Generate facts about "
        str += text + "in one sentence"
        generated_text += model(str, max_length=50, do_sample=True)[0]['generated_text']
    return generated_text


In [33]:
import jellyfish
import numpy as np

# Function to calculate Jaro-Winkler similarity
def jaro_winkler_similarity(str1, str2):
    return jellyfish.jaro_winkler_similarity(str1, str2)

# Function to calculate correlation
def calculate_correlation(cover_text, stego_text):
    min_length = min(len(cover_text), len(stego_text))
    cover_values = [ord(c) for c in cover_text[:min_length]]
    stego_values = [ord(c) for c in stego_text[:min_length]]
    return np.corrcoef(cover_values, stego_values)[0, 1]

# Function to embed the secret message into the cover text using specific characters
def embed_secret_message(cover_text, secret_message):
    cover_chars = list(cover_text)
    secret_chars = list(secret_message)

    # Start with the last character of the cover text
    unique_chars = []
    for char in reversed(cover_chars):
        if char not in unique_chars:
            unique_chars.append(char)

    # Embed the secret message
    secret_index = 0
    for char in unique_chars:
        for index, cover_char in enumerate(cover_chars):
            if cover_char == char and secret_index < len(secret_chars):
                cover_chars[index] = chr((ord(cover_chars[index]) ^ ord(secret_chars[secret_index])) % 256)
                secret_index += 1
        if secret_index >= len(secret_chars):
            break

    if secret_index < len(secret_chars):
        raise ValueError("Not enough characters to embed the secret message.")

    return ''.join(cover_chars).replace('\n', '')

# Function to extract the secret message from the stego text
def extract_secret_message(stego_text, cover_text, secret_length):
    stego_chars = list(stego_text)
    cover_chars = list(cover_text)

    # Start with the last character of the cover text
    unique_chars = []
    for char in reversed(cover_chars):
        if char not in unique_chars:
            unique_chars.append(char)

    secret_chars = []
    secret_index = 0
    for char in unique_chars:
        for index, cover_char in enumerate(cover_chars):
            if cover_char == char and secret_index < secret_length:
                secret_chars.append(chr((ord(stego_chars[index]) ^ ord(cover_chars[index])) % 256))
                secret_index += 1
        if secret_index >= secret_length:
            break

    return ''.join(secret_chars)

# Function to get the cover text (you should implement this to get the actual cover text)
# def get_cover_text():
#     # Placeholder function to simulate getting cover text.
#     # Replace with actual implementation.
#     return "This is the cover text used for embedding the secret message. Ensure this text is sufficiently long and has enough spaces."

# Get cover text
cover_text = get_cover_text()
print("Cover Text: ", cover_text)

# Get secret message and password from user
msg = input("Enter secret message: ")
password = input("Create key: ")

# Global dictionaries
d_sc = {}  # Stego key to stego text
d_cc = {}  # Stego text to secret message

# Embed the secret message
try:
    ciphertxt = embed_secret_message(cover_text, msg)
    cover_steg_key = input("Create stego key: ")
    d_sc[cover_steg_key] = ciphertxt  # Store stego text with stego key
    d_cc[ciphertxt] = msg  # Store secret message with stego text

    # Calculate and print Jaro-Winkler similarity
    jaro_winkler = jaro_winkler_similarity(cover_text, ciphertxt)
    print(f"Jaro-Winkler Similarity: {jaro_winkler:.4f}")

    # Calculate and print correlation
    correlation = calculate_correlation(cover_text, ciphertxt)
    print(f"Correlation: {correlation:.4e}")

    # Print Stego Object (for clarity)
    print("Stego Object: ", ciphertxt)
except ValueError as e:
    print(e)
    exit()

# Ask user if they want to reveal the message
reveal = input("Do you want to reveal the message? (y/n) ")
if reveal == 'y':
    # Function to get the stego object and key for revealing the message
    def steg_ci():
        count = 0
        while count < 5:
            cover_r = input("Enter Stego Object: ")
            skey = input("Enter stego key: ")
            if skey in d_sc.keys() and cover_r == d_sc[skey]:
                return d_sc[skey]
            else:
                print("Incorrect stego object or key")
                count += 1
        else:
            print("Exceeded limit")
            exit()

    # Reveal the secret message from the stego text
    def cipher_sec(stego):
        secret_length = len(msg)
        extracted_msg = extract_secret_message(stego, cover_text, secret_length)
        print("Secret message:", extracted_msg)

    stego = steg_ci()
    key = input("Enter Key: ")
    if key == password:
        cipher_sec(stego)
    else:
        print("Incorrect key")
        exit()


Cover Text:  Scientists study the functions of celestial bodies such as planets, stars, and galaxies.Science has revolutionized many areas of life, including medicine, engineering, and engineering.Science is the study of the universe created by natural phenomena.Science is the scientific study of matter and how it interacts with other substances in the natural world.Science is an abstract branch of physics that deals with the study of matter and forces that exist in the universe.Science is a branch of human biology that explains the origins of life and the processes that keep our planet functioning.- The theory of relativity states that the laws of physics are the same for all observers. - The atomic bomb in 1924 was dropped by a blimp, according to a theory developed by Albert EinsteinScience allows us to understand and explain numerous phenomena through our understanding of space and time.Science is a complex field of study that involves empirical evidence and observations about the 